In [ ]:
!pip install pandas
!pip install matplotlib
!pip install seaborn


**PREDICTING ACCIDENT SEVERITY**

In [ ]:
import pickle
import pandas as pd


Reading in Data

In [ ]:
accident_chain = pd.read_csv('ACCIDENT_CHAINAGE.csv')
accident_event =  pd.read_csv('ACCIDENT_EVENT.csv')
accident_location = pd.read_csv('ACCIDENT_LOCATION.csv')
accident = pd.read_csv('ACCIDENT.csv')
node_id_complex_int_id = pd.read_csv('NODE_ID_COMPLEX_INT_ID.csv')
node = pd.read_csv('NODE.csv')
person = pd.read_csv('PERSON.csv')
road_surface = pd.read_csv('ROAD_SURFACE_COND.csv')
subdca = pd.read_csv('SUBDCA.csv')
vehicle = pd.read_csv('VEHICLE.csv')

In [ ]:
accident

In [ ]:
#look at features
accident.columns

Only interested in Factors we can control and know before the crash e.g. Speed important but type of Accident not as it is only known after the fact

In [ ]:
accident['SPEED_ZONE'].value_counts()

In [ ]:
accident['Day Week Description'].value_counts()

In [ ]:
accident['Light Condition Desc'].value_counts()

In [ ]:
accident['Road Geometry Desc'].value_counts()

In [ ]:
accident['SEVERITY'].value_counts()

In [ ]:
# 1 is most severe and 4 is least severt
accident.loc[accident['SEVERITY']==1]

In [ ]:
#speed will be a determining factor so for all speeds greater than 110, remove the row
accident_filtered = accident[accident['SPEED_ZONE'] <= 110]


In [ ]:
type(person['SEATING_POSITION'][3]),person['SEATING_POSITION'][3]

In [ ]:
person.columns

In [ ]:
#interested in the driver as we will see if age/sex have any impact on severity of collission
drivers = person[person['SEATING_POSITION'] == 'D ']
drivers = drivers[['ACCIDENT_NO','SEX','AGE','HELMET_BELT_WORN']]

In [ ]:
drivers['HELMET_BELT_WORN'].value_counts()

In [ ]:
drivers['HELMET_BELT_WORN'] = drivers['HELMET_BELT_WORN'].astype(int)

In [ ]:
drivers['HELMET_BELT_WORN'].value_counts()

In [ ]:
drivers

In [ ]:
road_surface

In [ ]:
road_surface['Surface Cond Desc'].value_counts()

In [ ]:
vehicle.columns

In [ ]:
vehicle['VEHICLE_YEAR_MANUF'].value_counts()

easiest_date = vehicle['VEHICLE_YEAR_MANUF'].min()
latest_date = vehicle['VEHICLE_YEAR_MANUF'].max()

# Create a Series with the extracted dates
manufactured_dates = pd.Series({
    'Earliest Date Manufactured': easiest_date,
    'Latest Date Manufactured': latest_date
})
manufactured_dates.name = "Newest and Oldest Vehicles"
manufactured_dates_df = manufactured_dates.to_frame()

# Print the DataFrame with a nice format
print(manufactured_dates_df)


In [ ]:
# only keep cars manufactured between 1890 and 2020
vehicle_filtered = vehicle[(vehicle['VEHICLE_YEAR_MANUF'] >= 1890) & (vehicle['VEHICLE_YEAR_MANUF'] <= 2020)]
#turn year manufactured to be years old (years since 2020)
vehicle_filtered['VEHICLE_YEARS_OLD'] = 2020 - vehicle_filtered['VEHICLE_YEAR_MANUF']

In [ ]:
vehicle_filtered

In [ ]:
# Assuming Lamps are number of working headlights
vehicle_filtered['LAMPS'].value_counts()
vehicle_filtered.loc[vehicle_filtered['LAMPS']==1]

In [ ]:
accident_filtered.columns

Before Merging All DataFrames on ACCIDENT_NO, Remove All irrelevant columns to reduce dimensionality of data 

In [ ]:
accident_filtered = accident_filtered[['ACCIDENT_NO','ACCIDENTTIME','DAY_OF_WEEK','Day Week Description','LIGHT_CONDITION','Light Condition Desc','ROAD_GEOMETRY','Road Geometry Desc','SEVERITY','SPEED_ZONE']]

In [ ]:
road_surface.columns

In [ ]:
road_surface = road_surface[['ACCIDENT_NO','SURFACE_COND','Surface Cond Desc']]

In [ ]:
vehicle_filtered.columns

In [ ]:
vehicle_filtered = vehicle_filtered[['ACCIDENT_NO','VEHICLE_BODY_STYLE','TOTAL_NO_OCCUPANTS','VEHICLE_YEARS_OLD', 'LAMPS']]

In [ ]:
vehicle_filtered['TOTAL_NO_OCCUPANTS'].value_counts()

In [ ]:
vehicle_filtered.loc[vehicle_filtered['TOTAL_NO_OCCUPANTS']>20]

In [ ]:
#Going to Merge and Have one row Per Driver, this will have multiple rows per Incident

merged_data = drivers.merge(accident_filtered, on='ACCIDENT_NO', how='inner').merge(road_surface, on='ACCIDENT_NO', how='inner').merge(vehicle_filtered, on='ACCIDENT_NO', how='inner')
merged_data

In [ ]:
merged_data.columns

In [ ]:
merged_data = merged_data[merged_data['AGE'] > 0]

In [ ]:
merged_data['LAMPS'].value_counts()

In [ ]:
merged_data.to_pickle('crash_processed_data.pkl')


In [ ]:
drivers['HELMET_BELT_WORN'].value_counts()

In [ ]:
vehicle.loc[vehicle['LAMPS']==9,'VEHICLE_BODY_STYLE'].value_counts()